In [1]:
import os  
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1" 

import keras
from keras.datasets import cifar10
from keras import backend as K
from keras.layers import Input, Conv2D,GlobalAveragePooling2D, Dense, BatchNormalization, Activation
from keras.models import Model
from keras.layers import DepthwiseConv2D

from keras import optimizers,regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import he_normal
from keras.callbacks import LearningRateScheduler, TensorBoard, ModelCheckpoint

num_classes        = 10
batch_size         = 64         # 64 or 32 or other
epochs             = 300
iterations         = 782       
USE_BN=True
DROPOUT=0.2 # keep 80%
CONCAT_AXIS=3
weight_decay=1e-4
DATA_FORMAT='channels_last' # Theano:'channels_first' Tensorflow:'channels_last'

log_filepath  = './mobilenet_slim'

Using TensorFlow backend.


In [2]:
def color_preprocessing(x_train,x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    mean = [125.307, 122.95, 113.865]
    std  = [62.9932, 62.0887, 66.7048]
    for i in range(3):
        x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
        x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]
    return x_train, x_test

def scheduler(epoch):
    if epoch < 100:
        return 0.01
    if epoch < 200:
        return 0.001
    return 0.0001

# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test  = keras.utils.to_categorical(y_test, num_classes)
x_train, x_test = color_preprocessing(x_train, x_test)

In [3]:
def depthwise_separable(x,params):
    # f1/f2 filter size, s1 stride of conv
    (s1,f2) = params
    x = DepthwiseConv2D((3,3),strides=(s1[0],s1[0]), padding='same',depthwise_initializer="he_normal")(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = Conv2D(int(f2[0]), (1,1), strides=(1,1), padding='same',
               kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    return x

In [4]:
def MobileNet(img_input,shallow=False, classes=10):
    """Instantiates the MobileNet.Network has two hyper-parameters
        which are the width of network (controlled by alpha)
        and input size.
        # Arguments
            alpha: optional parameter of the network to change the 
                width of model.
            shallow: optional parameter for making network smaller.
            classes: optional number of classes to classify images
                into.
    """
    # change stride
    x = Conv2D(int(32), (3,3), strides=(1,1), padding='same',
              kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(weight_decay))(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = depthwise_separable(x,params=[(1,),(64,)])
    x = depthwise_separable(x,params=[(1,),(128,)])# change stride
    x = depthwise_separable(x,params=[(1,),(128,)])
    x = depthwise_separable(x,params=[(1,),(256,)])# change stride
    x = depthwise_separable(x,params=[(1,),(256,)])
    x = depthwise_separable(x,params=[(2,),(512,)])
    if not shallow:
        for _ in range(5):
            x = depthwise_separable(x,params=[(1,),(512,)])
            
    x = depthwise_separable(x,params=[(2,),(1024,)])
    x = depthwise_separable(x,params=[(1,),(1024,)])

    x = GlobalAveragePooling2D()(x)
    out = Dense(classes, activation='softmax')(x)
    return out

In [5]:
img_input=Input(shape=(32,32,3))
output = MobileNet(img_input)
model=Model(img_input,output)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
depthwise_conv2d_1 (Depthwis (None, 32, 32, 32)        320       
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
__________

In [6]:
# set optimizer
sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# set callback
tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=0)
change_lr = LearningRateScheduler(scheduler)
cbks = [change_lr,tb_cb]

# set data augmentation
datagen = ImageDataGenerator(horizontal_flip=True,
                             width_shift_range=0.125,
                             height_shift_range=0.125,
                             fill_mode='constant',cval=0.)
datagen.fit(x_train)

# start training
model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                    steps_per_epoch=iterations,
                    epochs=epochs,
                    callbacks=cbks,
                    validation_data=(x_test, y_test))
model.save('mobilenet_slim.h5')

Epoch 1/300
782/782 [==============================] - 99s 127ms/step - loss: 2.5424 - acc: 0.3997 - val_loss: 2.2428 - val_acc: 0.5149
Epoch 2/300
782/782 [==============================] - 89s 113ms/step - loss: 2.1159 - acc: 0.5626 - val_loss: 2.0601 - val_acc: 0.5926
Epoch 3/300
782/782 [==============================] - 89s 113ms/step - loss: 1.8293 - acc: 0.6633 - val_loss: 1.8190 - val_acc: 0.6806
Epoch 4/300
782/782 [==============================] - 89s 114ms/step - loss: 1.6318 - acc: 0.7304 - val_loss: 1.6853 - val_acc: 0.7269
Epoch 5/300
782/782 [==============================] - 89s 114ms/step - loss: 1.4922 - acc: 0.7730 - val_loss: 1.5348 - val_acc: 0.7714
Epoch 6/300
782/782 [==============================] - 89s 114ms/step - loss: 1.3988 - acc: 0.7986 - val_loss: 1.4508 - val_acc: 0.7929
Epoch 7/300
782/782 [==============================] - 89s 114ms/step - loss: 1.3156 - acc: 0.8213 - val_loss: 1.3690 - val_acc: 0.8105
Epoch 8/300
782/782 [===========================

Epoch 61/300
782/782 [==============================] - 89s 114ms/step - loss: 0.3651 - acc: 0.9762 - val_loss: 0.7419 - val_acc: 0.8995
Epoch 62/300
782/782 [==============================] - 92s 118ms/step - loss: 0.3544 - acc: 0.9782 - val_loss: 0.7693 - val_acc: 0.8933
Epoch 63/300
782/782 [==============================] - 89s 114ms/step - loss: 0.3571 - acc: 0.9758 - val_loss: 0.7736 - val_acc: 0.8930
Epoch 64/300
782/782 [==============================] - 89s 114ms/step - loss: 0.3417 - acc: 0.9804 - val_loss: 0.7316 - val_acc: 0.9030
Epoch 65/300
782/782 [==============================] - 89s 114ms/step - loss: 0.3397 - acc: 0.9797 - val_loss: 0.7107 - val_acc: 0.9016
Epoch 66/300
782/782 [==============================] - 89s 114ms/step - loss: 0.3396 - acc: 0.9789 - val_loss: 0.7533 - val_acc: 0.8929
Epoch 67/300
782/782 [==============================] - 89s 114ms/step - loss: 0.3306 - acc: 0.9802 - val_loss: 0.7467 - val_acc: 0.8959
Epoch 68/300
782/782 [===================

782/782 [==============================] - 89s 114ms/step - loss: 0.1925 - acc: 0.9995 - val_loss: 0.5418 - val_acc: 0.9275
Epoch 121/300
782/782 [==============================] - 89s 114ms/step - loss: 0.1918 - acc: 0.9995 - val_loss: 0.5409 - val_acc: 0.9269
Epoch 122/300
782/782 [==============================] - 89s 114ms/step - loss: 0.1908 - acc: 0.9997 - val_loss: 0.5467 - val_acc: 0.9259
Epoch 123/300
782/782 [==============================] - 89s 114ms/step - loss: 0.1903 - acc: 0.9997 - val_loss: 0.5449 - val_acc: 0.9249
Epoch 124/300
782/782 [==============================] - 89s 114ms/step - loss: 0.1896 - acc: 0.9996 - val_loss: 0.5429 - val_acc: 0.9272
Epoch 125/300
782/782 [==============================] - 89s 114ms/step - loss: 0.1891 - acc: 0.9995 - val_loss: 0.5392 - val_acc: 0.9260
Epoch 126/300
782/782 [==============================] - 90s 115ms/step - loss: 0.1887 - acc: 0.9994 - val_loss: 0.5432 - val_acc: 0.9273
Epoch 127/300
782/782 [=========================

782/782 [==============================] - 89s 113ms/step - loss: 0.1586 - acc: 0.9998 - val_loss: 0.5346 - val_acc: 0.9272
Epoch 180/300
782/782 [==============================] - 89s 114ms/step - loss: 0.1580 - acc: 0.9999 - val_loss: 0.5385 - val_acc: 0.9265
Epoch 181/300
782/782 [==============================] - 89s 114ms/step - loss: 0.1575 - acc: 0.9998 - val_loss: 0.5368 - val_acc: 0.9263
Epoch 182/300
782/782 [==============================] - 89s 114ms/step - loss: 0.1570 - acc: 0.9998 - val_loss: 0.5374 - val_acc: 0.9261
Epoch 183/300
782/782 [==============================] - 92s 118ms/step - loss: 0.1566 - acc: 0.9998 - val_loss: 0.5364 - val_acc: 0.9275
Epoch 184/300
782/782 [==============================] - 89s 114ms/step - loss: 0.1561 - acc: 0.9998 - val_loss: 0.5336 - val_acc: 0.9259
Epoch 185/300
782/782 [==============================] - 89s 113ms/step - loss: 0.1554 - acc: 0.9999 - val_loss: 0.5406 - val_acc: 0.9261
Epoch 186/300
782/782 [=========================

782/782 [==============================] - 89s 114ms/step - loss: 0.1463 - acc: 0.9999 - val_loss: 0.5269 - val_acc: 0.9278
Epoch 239/300
782/782 [==============================] - 89s 114ms/step - loss: 0.1462 - acc: 1.0000 - val_loss: 0.5210 - val_acc: 0.9283
Epoch 240/300
782/782 [==============================] - 89s 114ms/step - loss: 0.1462 - acc: 0.9999 - val_loss: 0.5239 - val_acc: 0.9284
Epoch 241/300
782/782 [==============================] - 89s 114ms/step - loss: 0.1461 - acc: 0.9999 - val_loss: 0.5232 - val_acc: 0.9277
Epoch 242/300
782/782 [==============================] - 89s 114ms/step - loss: 0.1461 - acc: 1.0000 - val_loss: 0.5231 - val_acc: 0.9269
Epoch 243/300
782/782 [==============================] - 89s 114ms/step - loss: 0.1460 - acc: 0.9999 - val_loss: 0.5220 - val_acc: 0.9274
Epoch 244/300
782/782 [==============================] - 89s 114ms/step - loss: 0.1460 - acc: 0.9999 - val_loss: 0.5286 - val_acc: 0.9286
Epoch 245/300
782/782 [=========================

782/782 [==============================] - 89s 114ms/step - loss: 0.1435 - acc: 1.0000 - val_loss: 0.5228 - val_acc: 0.9282
Epoch 298/300
782/782 [==============================] - 89s 114ms/step - loss: 0.1437 - acc: 0.9999 - val_loss: 0.5223 - val_acc: 0.9283
Epoch 299/300
782/782 [==============================] - 89s 114ms/step - loss: 0.1436 - acc: 0.9999 - val_loss: 0.5255 - val_acc: 0.9279
Epoch 300/300
782/782 [==============================] - 89s 114ms/step - loss: 0.1435 - acc: 0.9999 - val_loss: 0.5227 - val_acc: 0.9282
